In [ ]:
import pandas as pd
import os
from datetime import datetime, timedelta
from glob import glob

In [ ]:
from tqdm import tqdm # Progress bar
tqdm.pandas()

In [ ]:
flight_folder = "D:/STAT628m3/flight_holidayseanson/flight"
climate_folders = ['D:/STAT628m3/climate2018', 'D:/STAT628m3/climate2019', 'D:/STAT628m3/climate2020', 
                   'D:/STAT628m3/climate2021', 'D:/STAT628m3/climate2022', 'D:/STAT628m3/climate2023',
                   'D:/STAT628m3/climate2024']
output_folder = 'D:/STAT628m3'
# output_folder_extracted = 'D:/STAT628m3/extracted_data'
# output_folder_merged = 'D:/STAT628m3/merged_data'
output_file = 'D:/STAT628m3/flight_with_weather.csv'
# station_mapping_file = "D:/STAT628m3/Station_final_match.csv"

First, delete the line that climate is not from November to January (some have problems, but how much can be deleted, and the later time is faster)   
Direct overlay

In [ ]:
for folder in climate_folders:
    climate_files = glob(os.path.join(folder, '*.csv'))
    for file in climate_files:
        try:
            chunks = pd.read_csv(file, chunksize=1000, on_bad_lines='skip')
            climate_data = pd.concat(chunks, ignore_index=True)

            # Keep records from November to January
            climate_data['DATE'] = pd.to_datetime(climate_data['DATE'], errors='coerce')
            climate_data = climate_data[climate_data['DATE'].dt.month.isin([11, 12, 1])]
            
            climate_data.to_csv(file, index=False)
            print(f"Processed file: {file}")

        except Exception as e:
            print(f"Error processing file {file}: {e}")

In [ ]:
# Create output folder
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:
flight_columns = [
    "Year", "Month", "DayofMonth", "DayOfWeek", "Operating_Airline ", "Origin", "OriginCityName", "OriginState",
    "Dest", "DestCityName", "DestState", "CRSDepTime", "DepTime", "DepDelay", "DepTimeBlk", "CRSArrTime",
    "ArrTime", "ArrDelay", "ArrTimeBlk", "Cancelled", "CancellationCode", "Diverted", "CRSElapsedTime",
    "ActualElapsedTime", "Distance", "WeatherDelay"
]

climate_columns = [
    'DATE', 'REPORT_TYPE', 'SOURCE', 'HourlyAltimeterSetting', 'HourlyDewPointTemperature', 
    'HourlyDryBulbTemperature', 'HourlyPrecipitation', 'HourlyPresentWeatherType', 'HourlyPressureChange',
    'HourlyPressureTendency', 'HourlyRelativeHumidity', 'HourlySkyConditions', 'HourlySeaLevelPressure',
    'HourlyStationPressure', 'HourlyVisibility', 'HourlyWetBulbTemperature', 'HourlyWindDirection',
    'HourlyWindGustSpeed', 'HourlyWindSpeed'
]

Start by merging flight data

In [ ]:
flight_files = glob(os.path.join(flight_folder, '*.csv'))
flight_data = pd.concat([pd.read_csv(f) for f in flight_files], ignore_index=True)
flight_data = flight_data[flight_columns]

In [ ]:
flight_data.head()

In [ ]:
flight_data.shape

In [ ]:
# Write to file
flight_data.to_csv('D:/STAT628m3/flight_data.csv', index=False)